In [2]:
import pandas as pd

## Data imports

# Macro study of the HK listed equities market

### Macroeconomic indicators / exogenous variables

Data frequency is daily, monthly or quarterly. I will regress the daily indicators against daily returns of the HKEX / sectors and repeat similarly for other time frames. 

In [3]:
AUDUSD = pd.read_csv('data/AUD_USD.csv', index_col=0, parse_dates=True, infer_datetime_format=True)
AUDCNY = pd.read_csv('data/AUD_CNY.csv', index_col=0, parse_dates=True, infer_datetime_format=True)

CAIXIN = pd.read_csv('data/Caixin_PMI.csv', index_col=0, parse_dates=True, infer_datetime_format=True)
CN10Y = pd.read_csv('data/CN_10Y.csv', index_col=0, parse_dates=True, infer_datetime_format=True)
CN_GDP = pd.read_csv('data/CN_nominal_GDP.csv', index_col=0, parse_dates=True, infer_datetime_format=True)
CN_CPI = pd.read_csv('data/CN_10Y.csv', index_col=0, parse_dates=True, infer_datetime_format=True)

US10Y = pd.read_csv('data/US_10Y.csv', index_col=0, parse_dates=True, infer_datetime_format=True)
DXY = pd.read_csv('data/DXY.csv', index_col=0, parse_dates=True, infer_datetime_format=True)

### Quarterly comparisons to do
- CAIXIN, CN_GDP, US_GDP vs HK50 (indexed monthly)

### Daily comparisons to do
- AUDUSD, AUDCNY, DXY, CN10Y/US10Y differential (adjusted for inflation i.e. real rates) vs HK50 (indexed daily)
- each of the individual HSC indices vs HK50 overall. 

In [11]:
DXY.tail()

,Price,Open,High,Low,Vol.,Change %
Date,,,,,,
2000-01-07,100.80,100.49,100.93,100.44,NaN,0.15%
2000-01-06,100.65,100.31,100.81,99.81,NaN,0.27%
2000-01-05,100.38,100.42,100.47,99.71,NaN,-0.03%
2000-01-04,100.41,100.55,100.86,100.01,NaN,0.19%
2000-01-03,100.22,101.67,101.83,100.19,NaN,-1.62%


import HKEX / sector indices

In [4]:
HK50 = pd.read_csv('data/HS/HK50.csv', index_col=0, parse_dates=True, infer_datetime_format=True)
HSCC = pd.read_csv('data/HS/HSCC.csv', index_col=0, parse_dates=True, infer_datetime_format=True)
HSCE = pd.read_csv('data/HS/HSCE.csv', index_col=0, parse_dates=True, infer_datetime_format=True)
HSCIC = pd.read_csv('data/HS/HSCIC.csv', index_col=0, parse_dates=True, infer_datetime_format=True)
HSCICD = pd.read_csv('data/HS/HSCICD.csv', index_col=0, parse_dates=True, infer_datetime_format=True)
HSCICS = pd.read_csv('data/HS/HSCICS.csv', index_col=0, parse_dates=True, infer_datetime_format=True)
HSCIE = pd.read_csv('data/HS/HSCIE.csv', index_col=0, parse_dates=True, infer_datetime_format=True)
HSCIF = pd.read_csv('data/HS/HSCIF.csv', index_col=0, parse_dates=True, infer_datetime_format=True)
HSCIH = pd.read_csv('data/HS/HSCIH.csv', index_col=0, parse_dates=True, infer_datetime_format=True)
HSCIIG = pd.read_csv('data/HS/HSCIIG.csv', index_col=0, parse_dates=True, infer_datetime_format=True)
HSCIIT = pd.read_csv('data/HS/HSCIIT.csv', index_col=0, parse_dates=True, infer_datetime_format=True)
HSCIM = pd.read_csv('data/HS/HSCIM.csv', index_col=0, parse_dates=True, infer_datetime_format=True)
HSCIPC = pd.read_csv('data/HS/HSCIPC.csv', index_col=0, parse_dates=True, infer_datetime_format=True)
HSCIT = pd.read_csv('data/HS/HSCIT.csv', index_col=0, parse_dates=True, infer_datetime_format=True)
HSCIU = pd.read_csv('data/HS/HSCIU.csv', index_col=0, parse_dates=True, infer_datetime_format=True)

In [5]:
HK50.tail()

,Price,Open,High,Low,Vol.,Change %
Date,,,,,,
2000-12-11,15408.54,15274.07,15438.89,15193.49,NaN,1.44%
2000-12-08,15189.33,15008.80,15281.81,14968.64,NaN,1.18%
2000-12-07,15011.52,14982.92,15032.27,14839.38,NaN,-0.58%
2000-12-06,15098.95,14914.54,15178.27,14914.54,NaN,3.61%
2000-12-05,14573.21,14602.25,14701.98,14507.50,NaN,0.10%


If NaNs are present in the data, we should put a number to fix that. To prevent look-ahead bias, we used the fill-forward method and assume that it doesn't introduce extra issues associated with data granularity.

data processing: create a function that computes returns for relevant time series. How to deal with quarterly-indexed time series? Maybe to compare, I'll need code to reindex the daily time series.

To obtain a better understand of the macroeconomic dependence of the HK market, we will perform linear regression on a variety of indicators. And for each HKEX sector. Regression window will be 1M, 3M and 6M to identify regime changes over time.